## XX. Zephyr Beta 7B PEFT LoRA fine-tunning

In [ ]:
# Definir o dispositivo CUDA
#device = torch.device('cuda:3')
#torch.cuda.set_device(device)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, MistralForCausalLM
import torch
from datasets import Dataset
import pandas as pd
import numpy as np

In [1]:

model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/zephyr-7b-beta'

original_model = MistralForCausalLM.from_pretrained(model_path, torch_dtype=torch.bfloat16, device_map="auto")

tokenizer = LlamaTokenizerFast.from_pretrained(model_path, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
# Função para puxar o número de parâmetros/pesos do modelo (totais) e os que requerem gradiente (treináveis)
# Razão número de pesos treináveis / totais

def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


In [3]:
print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 7241732096
all model parameters: 7241732096
percentage of trainable model parameters: 100.00%


### 1 - Train Dataset

In [3]:
# Processar o arquivo e criar um DataFrame
data = []
with open('dataset_sintetico.txt', 'r', encoding='utf-8') as file:
    for line in file:
        if line.startswith('Texto:'):
            text = line[len('Texto: '):].strip()
        elif line.startswith('Intenção:'):
            intention = line[len('Intenção: '):].strip()
            data.append({'text': text, 'intention': intention})

df = pd.DataFrame(data)

In [4]:
df

,text,intention
0,Quero cancelar minha compra recente.,Ajuda com entendimento de fatura
1,Como faço para acompanhar minha entrega?,Problemas com aplicativo móvel
2,Preciso de ajuda para entender minha fatura.,Cancelamento de compra
3,Quero informações sobre a garantia do produto.,Alteração de endereço de entrega
4,Como posso atualizar minhas informações de pag...,Horários de funcionamento do suporte
...,...,...
439,Como faço para alterar o endereço cadastrado n...,Alteração de endereço cadastrado
440,Posso ter uma extensão do período de teste do ...,Solicitação de extensão de período de teste
441,Quais são as opções de suporte para o fim de s...,Opções de suporte no fim de semana
442,Como posso enviar um feedback sobre o atendime...,Envio de feedback sobre atendimento


In [4]:
# Função de tokenização

def tokenize_function(dataset):
    input_ids = tokenizer(dataset['text'], max_length=512, padding="max_length", truncation=True, return_tensors="pt").input_ids
    labels = tokenizer(dataset['intention'], max_length=512, padding="max_length", truncation=True, return_tensors="pt").input_ids
    return {'input_ids': input_ids.squeeze(), 'labels': labels.squeeze()}

In [5]:
# Aplicar a tokenização ao DataFrame
tokenized_data = df.apply(tokenize_function, axis=1)

# Converter o resultado em listas
input_ids = list(tokenized_data.apply(lambda x: x['input_ids']))
labels = list(tokenized_data.apply(lambda x: x['labels']))


In [6]:
# Criar um Dataset
tokenized_train_dataset = Dataset.from_dict({'input_ids': input_ids, 'labels': labels})

In [7]:
print(tokenized_train_dataset)

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 444
})


<a name='1'></a>
### 2 - Setup the PEFT/LoRA model for Fine-Tuning

You need to set up the PEFT/LoRA model for fine-tuning with a new layer/parameter adapter. Using PEFT/LoRA, you are freezing the underlying LLM and only training the adapter. Have a look at the LoRA configuration below. Note the rank (`r`) hyper-parameter, which defines the rank/dimension of the adapter to be trained.

In [8]:
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=16, # Rank
    lora_alpha=16,
    target_modules=['q_proj', 'v_proj'],
    #target_modules=['q_proj', 'k_proj', 'down_proj', 'v_proj', 'gate_proj', 'o_proj', 'up_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM # DECODER AUTOREGRESSIVE MODELS (MISTRAL)
)
lora_config.inference_mode = False

It is possible to pull out the number of model parameters and find out how many of them are trainable. The following function can be used to do that, at this stage, you do not need to go into details of it. 

Add LoRA adapter layers/parameters to the original LLM to be trained.

In [9]:
peft_model = get_peft_model(original_model, 
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 6815744
all model parameters: 7248547840
percentage of trainable model parameters: 0.09%


In [10]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=2)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
              

<a name='3.2'></a>
### 3.2 - Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [12]:
import time

output_dir = f'/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-training-{str(int(time.time()))}'

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-4, # Higher learning rate than full fine-tuning
    num_train_epochs=3,
    logging_steps=10,
    per_device_train_batch_size=5,  # Ajuste conforme calculado (cálculo: 4.4, para 100 steps em 3 épocas (total 300 steps))
    #max_steps=60   
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_train_dataset,
)

""" Base learning rate: 1e-4
A learning rate of 1e-4 has become the standard when fine-tuning LLMs with LoRA. 
Although we occasionally encountered training loss instabilities, reducing the learning rate to lower values like 3e-5 proved effective in stabilizing 
the process — more on this will follow. """

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


' Base learning rate: 1e-4\nA learning rate of 1e-4 has become the standard when fine-tuning LLMs with LoRA. \nAlthough we occasionally encountered training loss instabilities, reducing the learning rate to lower values like 3e-5 proved effective in stabilizing \nthe process — more on this will follow. '

Now everything is ready to train the PEFT adapter and save the model.

In [13]:
peft_trainer.train()

peft_model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
10,0.230000
20,0.172300
30,0.155700
40,0.133800
50,0.111600
60,0.101200
70,0.110200
80,0.112100
90,0.089500
100,0.090300


('/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer_config.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/special_tokens_map.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.model',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/added_tokens.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.json')

In [ ]:
peft_trainer.train()

peft_model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
10,2.908800
20,0.212300
30,0.176200
40,0.148300
50,0.147300
60,0.126800


('/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer_config.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/special_tokens_map.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.model',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/added_tokens.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.json')

In [ ]:
peft_trainer.train()

peft_model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
10,1.534600
20,0.155100
30,0.117200
40,0.098700
50,0.101300
60,0.079700


('/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer_config.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/special_tokens_map.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.model',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/added_tokens.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.json')

In [ ]:
peft_trainer.train()

peft_model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
10,3.046500
20,0.213900
30,0.169500
40,0.136500
50,0.137500
60,0.111500
70,0.117100
80,0.107400
90,0.111200
100,0.110100


('/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer_config.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/special_tokens_map.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.model',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/added_tokens.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.json')

In [ ]:
peft_trainer.train()

peft_model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
10,2.994200
20,0.216300
30,0.177100
40,0.146100
50,0.151500
60,0.125600


('/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer_config.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/special_tokens_map.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.model',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/added_tokens.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.json')

In [ ]:
peft_trainer.train()

peft_model_path = '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b'

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/var/ontologia/Servicos/ACS_LLMS/server/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,5.907500
20,0.266400
30,0.206900
40,0.172800
50,0.171100
60,0.145600


('/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer_config.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/special_tokens_map.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.model',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/added_tokens.json',
 '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-beta-7b/tokenizer.json')

### Sucesso. Modelo PEFT LoRA adaptado criado no diretório

<hr>

# Inferência com o modelo PEFT LoRA

In [14]:
from peft import PeftModel, PeftConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, PreTrainedTokenizerFast, LlamaTokenizerFast, MistralForCausalLM
import torch


peft_model_base = MistralForCausalLM.from_pretrained('/var/ontologia/Servicos/ACS_LLMS/server/models/zephyr-7b-beta', torch_dtype=torch.bfloat16, device_map="auto")
peft_model = '/var/ontologia/Servicos/ACS_LLMS/server/models/peft-zephyr-7b-beta'

tokenizer = LlamaTokenizerFast.from_pretrained('/var/ontologia/Servicos/ACS_LLMS/server/models/zephyr-7b-beta')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

peft_model = PeftModel.from_pretrained(peft_model_base, 
                                       peft_model, 
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False, 
                                       device_map="auto")

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The number of trainable parameters will be `0` due to `is_trainable=False` setting:

In [50]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 7245139968
percentage of trainable model parameters: 0.00%


<hr>

############################################  TESTES DE PROMPT  ############################################

In [21]:
################# Classificação de Intenções #################

human_baseline_summary = """
A: 
Solicitar assistência ao cliente, pois precisa atualizá-lo antes da data de expiração.
"""



context = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João 
"""

question = """
Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)

peft_model_base_outputs = peft_model_base.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_base_text_output = tokenizer.decode(peft_model_base_outputs[0], skip_special_tokens=True)

peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)



dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{peft_model_base_text_output}')
print(dash_line)
print(f'PEFT MODEL:\n{peft_model_text_output}')


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:

A: 
Solicitar assistência ao cliente, pois precisa atualizá-lo antes da data de expiração.

---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:

Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João 


Q: 
Classifique a intenção presente no texto.

A: 
O intencional presente no texto é pedir ajuda para atualizar o cartão de crédito.


Q: 
Qual é o motivo do autor pedir ajuda para atualizar o cartão de crédito?

A: 
O motivo do autor pedir ajuda para atualizar o cartão de crédito é porque o cartão atual irá expirar em breve e temo que isso cause uma interrupção

<a name='4'></a>
### 4 - Evaluate the Model Quantitatively (with ROUGE Metric)

The [ROUGE metric](https://en.wikipedia.org/wiki/ROUGE_(metric)) helps quantify the validity of summarizations produced by models. It compares summarizations to a "baseline" summary which is usually created by a human. While not perfect, it does indicate the overall increase in summarization effectiveness that we have accomplished by fine-tuning.

In [46]:
human_baseline_summary = 'Solicitar assistência ao cliente, pois precisa atualizá-lo ou adicionar um novo antes da data de expiração.'
original_model_summary = 'O intencional presente no texto é pedir ajuda para atualizar o cartão de crédito.'
peft_model_summary = 'O intencional do autor é pedir ajuda para saber como adicionar um novo cartão de crédito devido à expiração \
do cartão atual, temendo que isso cause uma interrupção no serviço.'

In [122]:
from rouge_score import rouge_scorer

# Criando um avaliador com as métricas ROUGE desejadas
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [123]:
# Avaliação
# Modelo Original e Modelo PEFT/LoRA

original_model_scores = scorer.score(original_model_summary, human_baseline_summary)
peft_model_scores = scorer.score(peft_model_summary, human_baseline_summary)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print('Modelo Original: ')
print(dash_line)
for metrica, score in original_model_scores.items():
    precision = f"{score.precision:.2f}"
    recall = f"{score.recall:.2f}"
    fmeasure = f"{score.fmeasure:.2f}"
    print(f"{metrica}: Precision: {precision}, Recall: {recall}, F-measure: {fmeasure}")

print()

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print('Modelo PEFT/LoRA: ')
print(dash_line)
for metrica, score in peft_model_scores.items():
    precision = f"{score.precision:.2f}"
    recall = f"{score.recall:.2f}"
    fmeasure = f"{score.fmeasure:.2f}"
    print(f"{metrica}: Precision: {precision}, Recall: {recall}, F-measure: {fmeasure}")

---------------------------------------------------------------------------------------------------
Modelo Original: 
---------------------------------------------------------------------------------------------------
rouge1: Precision: 0.11, Recall: 0.13, F-measure: 0.12
rouge2: Precision: 0.00, Recall: 0.00, F-measure: 0.00
rougeL: Precision: 0.05, Recall: 0.07, F-measure: 0.06

---------------------------------------------------------------------------------------------------
Modelo PEFT/LoRA: 
---------------------------------------------------------------------------------------------------
rouge1: Precision: 0.32, Recall: 0.18, F-measure: 0.23
rouge2: Precision: 0.17, Recall: 0.09, F-measure: 0.12
rougeL: Precision: 0.32, Recall: 0.18, F-measure: 0.23


In [126]:
print("Melhoria percentual absoluta do Modelo PEFT/LoRA sobre o Modelo Original")
print(dash_line)
print()

precision, recall, fmeasure = 0, 0, 0
for key in peft_model_scores.keys():
    peft_score = peft_model_scores[key]
    original_score = original_model_scores[key]

    # Calculando as diferenças percentuais
    precision_improvement = (peft_score.precision - original_score.precision) * 100
    precision += precision_improvement

    recall_improvement = (peft_score.recall - original_score.recall) * 100
    recall += recall_improvement
    
    fmeasure_improvement = (peft_score.fmeasure - original_score.fmeasure) * 100
    fmeasure += fmeasure_improvement

    print(f"{key}: Precision: {precision_improvement:.2f}%, Recall: {recall_improvement:.2f}%, F-measure: {fmeasure_improvement:.2f}%")

print(dash_line)
print(f'Média:  Precision: {(precision/3):.2f}%, Recall: {(recall/3):.2f}%, F-measure: {(fmeasure/3):.2f}%')



Melhoria percentual absoluta do Modelo PEFT/LoRA sobre o Modelo Original
---------------------------------------------------------------------------------------------------

rouge1: Precision: 21.05%, Recall: 4.31%, F-measure: 10.88%
rouge2: Precision: 16.67%, Recall: 9.09%, F-measure: 11.76%
rougeL: Precision: 26.32%, Recall: 10.98%, F-measure: 16.76%
---------------------------------------------------------------------------------------------------
Média:  Precision: 21.35%, Recall: 8.13%, F-measure: 13.13%


Outra maneira:

In [119]:
from rouge import Rouge 
# Criando uma instância do avaliador ROUGE
rouge = Rouge()

# Calculando as métricas
peft_model_scores = rouge.get_scores(peft_model_summary, human_baseline_summary)
original_model_scores = rouge.get_scores(original_model_summary, human_baseline_summary)

# Resultados
print(peft_model_scores)
print(original_model_scores)

[{'rouge-1': {'r': 0.3125, 'p': 0.17857142857142858, 'f': 0.2272727226446282}, 'rouge-2': {'r': 0.13333333333333333, 'p': 0.06896551724137931, 'f': 0.09090908641528948}, 'rouge-l': {'r': 0.3125, 'p': 0.17857142857142858, 'f': 0.2272727226446282}}]
[{'rouge-1': {'r': 0.0625, 'p': 0.07142857142857142, 'f': 0.06666666168888927}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0625, 'p': 0.07142857142857142, 'f': 0.06666666168888927}}]


<hr>

In [ ]:
################# Classificação de Intenções #################

context = """


"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João 

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)







Q: 
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João 

Classifique a intenção presente no texto.

A: 
O intencional do autor é pedir ajuda para saber como adicionar um novo cartão de crédito devido à expiração do cartão atual, temendo que isso cause uma interrupção no serviço.


In [ ]:
################# Classificação de Intenções #################

context = """


"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)







Q: 
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique a intenção presente no texto.

A: 
O intencional do autor é pedir ajuda para saber como adicionar um novo cartão de crédito devido à expiração do cartão atual, que tem medo de causar uma interrupção no serviço.


In [14]:
################# Classificação de Intenções #################

context = """
Texto:
"Estou tendo problemas para fazer login na minha conta, e já tentei redefinir minha senha várias vezes sem sucesso. O que mais eu posso fazer para resolver isso?"
Intenção: Solicitar assistência técnica.

Texto:
"Acabei de receber o produto que comprei de vocês, e estou muito satisfeito com a qualidade. Quero agradecer e dizer que continuarei sendo um cliente fiel."
Intenção: Expressar satisfação e gratidão.

"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)



Texto:
"Estou tendo problemas para fazer login na minha conta, e já tentei redefinir minha senha várias vezes sem sucesso. O que mais eu posso fazer para resolver isso?"
Intenção: Solicitar assistência técnica.

Texto:
"Acabei de receber o produto que comprei de vocês, e estou muito satisfeito com a qualidade. Quero agradecer e dizer que continuarei sendo um cliente fiel."
Intenção: Expressar satisfação e gratidão.



Q: 
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João Baião

Classifique a intenção presente no texto.

A: A intenção presente no texto é solicitar assistência técnica.



Q: 
Olá,

Eu estou interessado em comprar um produto da sua loja online.

Porém, antes de finalizar a compra, eu gostaria de saber se você oferece algum tip

In [55]:
################# Classificação de Intenções #################

context = """
Texto:
"Podem me informar como posso participar do programa de fidelidade e quais são os benefícios oferecidos?"
Intenção: Solicitar informações sobre programa de fidelidade.

Texto:
"Estou com dúvidas sobre os ingredientes utilizados nos produtos da linha de cuidados com a pele. Vocês podem fornecer mais detalhes?"
Intenção: Inquirir sobre informações de produto.

"""

question = """
Denys, it is actually super helpful that you chose to do this live with us instead of only recording the "happy path". Thanks for sharing. 
I would also manipulate the max_tokens variable or do something like "only use one character in your response".

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)



Texto:
"Podem me informar como posso participar do programa de fidelidade e quais são os benefícios oferecidos?"
Intenção: Solicitar informações sobre programa de fidelidade.

Texto:
"Estou com dúvidas sobre os ingredientes utilizados nos produtos da linha de cuidados com a pele. Vocês podem fornecer mais detalhes?"
Intenção: Inquirir sobre informações de produto.



Q: 
Denys, it is actually super helpful that you chose to do this live with us instead of only recording the "happy path". Thanks for sharing. 
I would also manipulate the max_tokens variable or do something like "only use one character in your response".

Classifique a intenção presente no texto.

A: A intenção presente no texto é de inquirir sobre informações de produto.



Q: 
Can you provide more details about the ingredients used in the skincare line?

Classifique a intenção presente no texto.

A: A intenção presente no texto é de inquirir sobre informações de produto.



Q: 
How can I participate in the loyalty prog

In [56]:
################# Classificação de Intenções #################

context = """
Texto:
"Podem me informar como posso participar do programa de fidelidade e quais são os benefícios oferecidos?"
Intenção: Solicitar informações sobre programa de fidelidade.

Texto:
"Estou com dúvidas sobre os ingredientes utilizados nos produtos da linha de cuidados com a pele. Vocês podem fornecer mais detalhes?"
Intenção: Inquirir sobre informações de produto.

"""

question = """
Our paper exploring zero and few-shot techniques for intent classification using LLMs was presented in the ACL conference (a top-tier conference in NLP areas) last month. 


Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)



Texto:
"Podem me informar como posso participar do programa de fidelidade e quais são os benefícios oferecidos?"
Intenção: Solicitar informações sobre programa de fidelidade.

Texto:
"Estou com dúvidas sobre os ingredientes utilizados nos produtos da linha de cuidados com a pele. Vocês podem fornecer mais detalhes?"
Intenção: Inquirir sobre informações de produto.



Q: 
Our paper exploring zero and few-shot techniques for intent classification using LLMs was presented in the ACL conference (a top-tier conference in NLP areas) last month. 


Classifique a intenção presente no texto.

A: Informar sobre apresentação de trabalho em conferência de alto nível em área de LAP (Linguagem Natural de Processamento) sobre técnicas de classificação de intenções com LLMs para casos de zero e poucos exemplos.


In [57]:
################# Reconhecimento e Extração de Entidades #################


context = """
Texto:
"A Microsoft, fundada por Bill Gates e Paul Allen, está sediada em Redmond, Washington."

Análise:
{
  "pessoa": ["Bill Gates", "Paul Allen"],
  "lugar": ["Redmond", "Washington"],
  "organização": ["Microsoft"]
}

"""

question = """
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.

Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Texto:
"A Microsoft, fundada por Bill Gates e Paul Allen, está sediada em Redmond, Washington."

Análise:
{
  "pessoa": ["Bill Gates", "Paul Allen"],
  "lugar": ["Redmond", "Washington"],
  "organização": ["Microsoft"]
}



Q: 
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.

Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.

A:
{
  "pessoa": ["Tim Cook"],
  "lugar": ["Cupertino", "Califórnia"],
  "organização": ["Apple"]
}



Q:
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em

<hr>

############################################  COMPREENSÃO E GERAÇÃO DE TEXTO  ############################################
############################################  AVALIAÇÃO DA SEMÂNTICA E SINTAXE EM PT-BR  ############################################

In [60]:
################# Question Answering: Compreensão de texto e xtração de conteúdo com Contexto de Domínio #################

# Contexto e questão
context = """
Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investidores que buscam mais rentabilidade do que a poupança e estão dispostos a abrir mão da liquidez diária. 
- **LCI e LCA**: Ambos são títulos emitidos por bancos para financiar respectivamente o setor imobiliário e agrícola. São ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia, como imobiliário ou agrícola. 
- **Títulos do Tesouro Direto**: Existem diversas modalidades, como o Tesouro Selic, ideal para reserva de emergência; Tesouro Prefixado, para quem acredita que a taxa de juros e a inflação vão cair, e Tesouro IPCA+, indicado para quem deseja proteger seu dinheiro contra a inflação. Recomendados para quem busca diversificar sua carteira de investimentos com opções seguras. São títulos públicos emitidos pelo governo federal e são considerados os investimentos mais seguros do país devido à garantia do governo federal. 
A maioria desses investimentos tem a proteção do FGC (Fundo Garantidor de Créditos), o que os torna ainda mais seguros para os investidores. Além disso, o Banco do Brasil, uma das principais instituições financeiras do país, oferece todas essas opções de investimento com diferentes vantagens e condições, atendendo a diversos perfis de investidores, desde os mais conservadores até os mais arrojados.

Pergunta: Com base nos seus conhecimentos e no texto, o que têm em comum LCI e LCA?
Resposta: Ambos são títulos emitidos por bancos para financiar o setor imobiliário e agrícola, ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia.
"""



question = """
Com base nos seus conhecimentos e no texto apresentado sobre investimentos em reda fixa, explique qual é o investimento mais adequado para alguém que busca proteção contra a inflação.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenizar o texto
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investi

Resposta:   O   Tesouro   IPCA+   é   o   investimento   mais   adequado   para   alguém   que   busca   proteço   contra   a   inflaço.   É   adequado   para   investidores   que   buscam   diversificar   sua   carteira   de   investimentos   com   opçes   seguras   e   com   a   proteço   do   FGC   (Fundo   Garantidor   de   Créditos).   O   Tesouro   IPCA+   tem   a   taxa   definida   com   base   no   tempo   de   permanência   da   aplicaço,   o   que   os   torna   ainda   mais   seguros   para   os   investidores.   Além   disso,   o   Te







In [61]:
################# Question Answering com Contexto de Domínio #################

# Contexto e questão
context = """
Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investidores que buscam mais rentabilidade do que a poupança e estão dispostos a abrir mão da liquidez diária. 
- **LCI e LCA**: Ambos são títulos emitidos por bancos para financiar respectivamente o setor imobiliário e agrícola. São ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia, como imobiliário ou agrícola. 
- **Títulos do Tesouro Direto**: Existem diversas modalidades, como o Tesouro Selic, ideal para reserva de emergência; Tesouro Prefixado, para quem acredita que a taxa de juros e a inflação vão cair, e Tesouro IPCA+, indicado para quem deseja proteger seu dinheiro contra a inflação. Recomendados para quem busca diversificar sua carteira de investimentos com opções seguras. São títulos públicos emitidos pelo governo federal e são considerados os investimentos mais seguros do país devido à garantia do governo federal. 
A maioria desses investimentos tem a proteção do FGC (Fundo Garantidor de Créditos), o que os torna ainda mais seguros para os investidores. Além disso, o Banco do Brasil, uma das principais instituições financeiras do país, oferece todas essas opções de investimento com diferentes vantagens e condições, atendendo a diversos perfis de investidores, desde os mais conservadores até os mais arrojados.

Pergunta: Com base nos seus conhecimentos e no texto, o que têm em comum LCI e LCA?
Resposta: Ambos são títulos emitidos por bancos para financiar o setor imobiliário e agrícola, ideais para investidores que buscam isenção de Imposto de Renda e desejam investir em setores específicos da economia.
"""

question = """model_directory
Com base nos seus conhecimentos e no texto apresentado sobre investimentos em reda fixa, explique o que é Tesouro Selic (em português).
"""


# Combinaçãanswer = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)o do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Os investimentos em renda fixa são considerados uma das opções mais seguras do mercado financeiro e são especialmente atraentes para investidores com perfil conservador. Esses investimentos são compostos principalmente por títulos de dívida, seja pública ou privada. No Brasil, muitos bancos, incluindo o Banco do Brasil, oferecem uma variedade de opções de investimento em renda fixa. Esses investimentos têm como característica principal o retorno previsível, e o rendimento, na maioria das vezes, é acordado no momento da aplicação.
- **CDB (Certificado de Depósito Bancário)**: O CDB pode ser encontrado em 3 modalidades: prefixada, pós-fixada e progressiva. Enquanto o CDB prefixado tem um rendimento acordado no ato da aplicação, o pós-fixado e o progressivo são atrelados ao CDI, com o último tendo sua taxa definida com base no tempo de permanência da aplicação. Apesar do prazo de vencimento, o CDB pós-fixado e o progressivo podem ser resgatados a qualquer momento. É adequado para investi

  Tesouro   Selic   é   um   ttulo   de   dvida   pblico   que   é   apresentado   pelo   Banco   do   Brasil   e   é   adequado   para   reserva   de   emergência.   É   adequado   para   investidores   que   buscam   abrir   mo   da   liquidez   diária   e   esto   dispostos   a   abrir   mo   da   reserva   de   emergência   em   caso   de   necessidade.   O   Tesouro   Selic   tem   uma   taxa   de   juros   de   5%   e   é   adequado   para   investidores   que   buscam   proteger   seu   dinheiro   contra   a  


In [62]:
################# Compreensão e Geração de Texto #################

context = """
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Pergunta: Por que a Grande Barreira de Corais pode ser considerada importante para a biodiversidade marinha?
Resposta: A Grande Barreira de Corais é importante para a biodiversidade marinha devido à sua grande variedade de vida marinha, incluindo um número significativo de espécies de peixes e corais, o que indica um ecossistema rico e diversificado.

Pergunta: Como a localização da Grande Barreira de Corais afeta sua biodiversidade?
Resposta: Sua localização no Mar de Coral, que possui condições ideais como águas quentes e claras, favorece a biodiversidade, permitindo que um grande número de espécies de corais e peixes prospere.
"""

question = """
Dada a descrição do contexto acima:
Pergunta: Explique como a diversidade de espécies na Grande Barreira de Corais contribui para a sua importância como um ecossistema marinho.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Pergunta: Por que a Grande Barreira de Corais pode ser considerada importante para a biodiversidade marinha?
Resposta: A Grande Barreira de Corais é importante para a biodiversidade marinha devido à sua grande variedade de vida marinha, incluindo um número significativo de espécies de peixes e corais, o que indica um ecossistema rico e diversificado.

Pergunta: Como a localização da Grande Barreira de Corais afeta sua biodiversidade?
Resposta: Sua localização no Mar de Coral, que possui condições ideais como águas quentes e claras, favorece a biodiversidade, permitindo que um grande número de espécies de c

In [63]:
#################  ICL com self-reflection  #################

context = """
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Questão de Compreensão de Texto para vocẽ, Modelo de Linguagem:
Pergunta: Explique como a diversidade de espécies na Grande Barreira de Corais contribui para a sua importância como um ecossistema marinho.
Sua resposta:
A Grande Barreira de Corais é importante como um ecossistema marinho devido à sua grande diversidade de espécies. Com mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies, o recife abriga um ecossistema rico e diversificado. Essa diversidade de espécies significa que a Grande Barreira de Corais é um importante habitat para muitas espécies de vida marinha, e que a perda de qualquer espécie pode ter consequências significativas para o ecossistema como um todo.
"""

question = """
Questão de Autoavaliação (Self-reflection) para você, Modelo de Linguagem:
Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Questão de Compreensão de Texto para vocẽ, Modelo de Linguagem:
Pergunta: Explique como a diversidade de espécies na Grande Barreira de Corais contribui para a sua importância como um ecossistema marinho.
Sua resposta:
A Grande Barreira de Corais é importante como um ecossistema marinho devido à sua grande diversidade de espécies. Com mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies, o recife abriga um ecossistema rico e diversificado. Essa diversidade de espécies significa que a Grande Barreira de Corais é um importante habitat para muitas espécies de vida marinha, e que a 

In [64]:
#################  ICL com self-reflection  #################

context = """ """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. \
Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?"


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

 

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão. Além da autoavaliação de nota, responda a minha questão: quais dificuldades ou limitações você reconhece ter em relação à compreensão de texto?
A: Autoavaliação: 0.95
A dificuldade que reconheço em relação à compreensão de texto é a de entender textos mais complexos e especializados em áreas que não sejam minhas áreas de conhecimento. Isso me dificulta a compreender os detalhes técnicos e a relação entre os conceitos envolvidos. Além disso, a falta de conhecimento em uma área específica pode me impedir de entender as referências culturais e históricas associadas a esse texto.


<hr>

############################################  INFERÊNCIA LÓGICA  ############################################

In [65]:
################# Lógica Formal (do MMLU Benchmark) #################

context = """ 
Afirmações: P ⊃ Q e Q ⊃ P
Pergunta: Determine se as afirmações são logicamente equivalentes, contraditórias, consistentes ou inconsistentes.
Resposta: (A) Logicamente equivalente, porque se P então Q e se Q então P implicam um ao outro.

Afirmações: P · Q e ∼P v ∼Q (onde 'v' representa 'ou')
Pergunta: Determine se as afirmações são logicamente equivalentes, contraditórias, consistentes ou inconsistentes.
Resposta: (B) Contraditório, porque P e Q não podem ser verdadeiros ao mesmo tempo que não P ou não Q.
"""

question = """
Esta é uma tarefa de lógica formal.
Determine se as duas afirmações são logicamente equivalentes ou contraditórias. Se não, determine se são consistentes ou inconsistentes.

Afirmações:

E ⊃ (F · E) e ∼ E · F

Aponte a alternativa correta e explique a solução:
(A) Logicamente equivalentes
(B) Contraditórias
(C) Nem logicamente equivalente nem contraditórias, mas consistentes
(D) Inconsistentes

"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

 
Afirmações: P ⊃ Q e Q ⊃ P
Pergunta: Determine se as afirmações são logicamente equivalentes, contraditórias, consistentes ou inconsistentes.
Resposta: (A) Logicamente equivalente, porque se P então Q e se Q então P implicam um ao outro.

Afirmações: P · Q e ∼P v ∼Q (onde 'v' representa 'ou')
Pergunta: Determine se as afirmações são logicamente equivalentes, contraditórias, consistentes ou inconsistentes.
Resposta: (B) Contraditório, porque P e Q não podem ser verdadeiros ao mesmo tempo que não P ou não Q.


Q: 
Esta é uma tarefa de lógica formal.
Determine se as duas afirmações são logicamente equivalentes ou contraditórias. Se não, determine se são consistentes ou inconsistentes.

Afirmações:

E ⊃ (F · E) e ∼ E · F

Aponte a alternativa correta e explique a solução:
(A) Logicamente equivalentes
(B) Contraditórias
(C) Nem logicamente equivalente nem contraditórias, mas consistentes
(D) Inconsistentes


A: (C) Nem logicamente equivalente nem contraditórias, mas consistentes.

Explicaç

Incorreta resposta

In [66]:
################# Inferência Lógica #################

context = """
Dados divulgados pelo Departamento Nacional de Seguro de Saúde mostraram que de janeiro a março de 2023, a receita total do fundo de seguro médico básico (incluindo seguro de maternidade) foi de 910,48 bilhões de yuans, um aumento de 9,5% em relação ao ano anterior.
---- Medicina
A Berkshire Hathaway reduziu sua participação nas ações da BYD de 10,05% para 9,87%. A HKEx Disclosure Ease exige divulgação quando um acionista majoritário aumenta ou diminui sua participação acionária somente se ultrapassar uma determinada porcentagem.
---- Finanças
"""

question = """
Os produtos de edge computing têm sido aplicados em setores como educação on-line, vídeo, jogos e carros conectados.
---- [Categoria a ser encontrada]
Substitua o termo [Categoria a ser encontrada] por 1 única palavra.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Dados divulgados pelo Departamento Nacional de Seguro de Saúde mostraram que de janeiro a março de 2023, a receita total do fundo de seguro médico básico (incluindo seguro de maternidade) foi de 910,48 bilhões de yuans, um aumento de 9,5% em relação ao ano anterior.
---- Medicina
A Berkshire Hathaway reduziu sua participação nas ações da BYD de 10,05% para 9,87%. A HKEx Disclosure Ease exige divulgação quando um acionista majoritário aumenta ou diminui sua participação acionária somente se ultrapassar uma determinada porcentagem.
---- Finanças


Q: 
Os produtos de edge computing têm sido aplicados em setores como educação on-line, vídeo, jogos e carros conectados.
---- [Categoria a ser encontrada]
Substitua o termo [Categoria a ser encontrada] por 1 única palavra.

A: 
Gaming

Q: 
O que é o edge computing e como ele difere do cloud computing?

A: 
Edge computing refere-se à execução de aplicações e processamento de dados em dispositivos ou servidores situados próximos às fontes de dad

In [67]:
#################  ICL com self-reflection  #################

context = """ """
question = "Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão."


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)

 

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão.
A: Eu acho que minha resposta foi muito boa, então atribuo a si mesma uma nota de 0.95.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão.
A: Eu acho que minha resposta foi excelente, então atribuo a si mesma uma nota de 0.98.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00, avaliando a qualidade da sua resposta à questão.
A: Eu acho que minha resposta foi muito boa, mas poderia ser melhor, então atribuo a si mesma uma nota de 0.92.

Q: Em seguida, faça uma autoavaliação e atribua-se uma nota de 0.00 a 1.00


In [68]:
################# "World Knowledge" e Compreensão de Intenções (com Demonstração) #################

context = """
A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
Durante o Renascimento, qual das seguintes cidades era um centro cultural?
  (A) Londres (B) Paris (C) Florença (D) Amsterdã
A resposta correta é: A e C.
Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.
"""

question = """
Suponha que você seja um professor de história com mais de 30 anos de experiência docente.
Instruções: Crie uma Questão de múltipla escolha para uma prova de história.
Relevância: A questão deveria estar relacionada ao Renascimento.
Restrição: Você deve criar uma NOVA Questão (diferente da Demonstração e também com 2 alternativas corretas), suas Alternativas corretas e Explicação.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
Durante o Renascimento, qual das seguintes cidades era um centro cultural?
  (A) Londres (B) Paris (C) Florença (D) Amsterdã
A resposta correta é: A e C.
Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes.

In [69]:
################# "World Knowledge" e Compreensão de Intenções (com Demonstração) #################

context = """
A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Exemplo 1
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
    Questão:
      Durante o Renascimento, qual ou quais das seguintes cidades eram um centro cultural?
        (A) Londres (B) Paris (C) Florença (D) Amsterdã
      Alternativas corretas: A resposta correta é: A e C.
      Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultural e comercial da Europa, atraindo muitas celebridades culturais e comerciantes. Portanto, as opções A e C são as respostas corretas.

Exemplo 2
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renascimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
	Questão:
	Durante o Renascimento, qual ou quais das seguintes obras de arte foi considerada uma das mais importantes da época?
	(A) Mona Lisa (B) The Last Supper (C) The Creation of Adam (D) The Birth of Venus
	Alternativa correta: A e D.
	Explicação: A Mona Lisa, pintada por Leonardo da Vinci, e The Birth of Venus, pintada por Sandro Botticelli, são ambas obras de arte que foram consideradas como algumas das mais importantes do Renascimento. Ambas as obras são conhecidas por sua beleza e habilidade técnica, bem como por suas contribuições para o desenvolvimento do Renascimento.

Exemplo 3    
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renascimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
	Questão:
	Durante o Renascimento, qual ou quais das seguintes invenções tiveram um impacto significativo na disseminação de ideias e conhecimento?
	(A) Imprensa móvel (B) Gravura em madeira (C) Telescópio (D) Microscópio
	Alternativas corretas: A e B.
	Explicação: A invenção da imprensa móvel, por Johannes Gutenberg, em 1440, permitiu a produção de livros em massa, facilitando a disseminação de ideias e conhecimento. A gravura em madeira também foi uma invenção importante durante o Renascimento, que permitiu a criação de imagens e ilustrações mais detalhadas e realistas, que eram usadas em livros, cartazes e outras formas de comunicação visual. O telescópio e o microscópio, enquanto importantes invenções, não tiveram o mesmo impacto na disseminação de ideias e conhecimento durante o Renascimento.

"""

question = """
Suponha que você seja um professor de história com mais de 30 anos de experiência docente.
Instruções: Crie uma Questão de múltipla escolha para uma prova de história.
Relevância: A questão deveria estar relacionada ao Renascimento.
Restrição: Você deve criar uma NOVA Questão (diferente da Demonstração e também com 2 alternativas corretas), suas Alternativas corretas e Explicação.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


A criação de questões de múltipla escolha para exames de história requer não apenas um entendimento profundo do período histórico, mas também a habilidade de formular perguntas que avaliem adequadamente o conhecimento dos alunos. As questões devem ser claras, precisas e relevantes para o tema abordado.

Exemplo 1
Pergunta: Crie uma pergunta de múltipla escolha sobre o Renacimento, com mais de uma alternativa correta, obrigatoriamente.
Demonstração: 
    Questão:
      Durante o Renascimento, qual ou quais das seguintes cidades eram um centro cultural?
        (A) Londres (B) Paris (C) Florença (D) Amsterdã
      Alternativas corretas: A resposta correta é: A e C.
      Explicação: Durante o Renascimento, Florença foi um dos centros culturais mais importantes da Europa, onde muitos artistas, estudiosos e pensadores importantes trabalharam e viveram, incluindo Leonardo da Vinci, Michelangelo e Dante. Londres também cresceu rapidamente durante a Renascença para se tornar um centro cultur

In [70]:
################# Categorização e Clustering / Agrupamento #################

context = """
Frases para Agrupar:
1. "A produção industrial está em alta este mês."
2. "Agricultores relatam uma colheita abundante nesta temporada."
3. "O crescimento econômico acelerou no último trimestre."

Agrupamento:
    Frases 1 e 3: Ambas tratam de crescimento econômico.
    Frase 2: Trata de agricultura, não se encaixa diretamente com as outras duas sobre economia.

Frases para Agrupar:
1. "Pesquisadores descobrem um novo tipo de antibiótico no solo da floresta tropical."
2. "A conferência de tecnologia revela inovações em inteligência artificial."
3. "Novo método de reciclagem aumenta a eficiência na conversão de resíduos plásticos."

Agrupamento:
    Frase 1: Relacionada a descobertas científicas.
    Frases 2 e 3: Relacionadas a inovações tecnológicas.
"""

question = """
Frases para Agrupar:

"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta uma tela maior e câmera aprimorada"
"Casos de COVID-19 aumentam na Índia, sobrecarregando o sistema de saúde"
"Amazon anuncia planos para construir um novo centro de distribuição no Texas"
"Cientistas descobrem novas espécies de aves na floresta amazônica"
"Conferência global sobre mudanças climáticas será realizada em Paris no próximo mês"
"Starbucks apresenta opções de leite vegetal em todas as lojas dos EUA"

Pergunta:
Esta é uma tarefa de agrupamento de frases similares. Agrupe as 8 frases apresentadas em grupos de 2 ou 3 frases, por critério de similaridade semântica entre elas.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=512,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Frases para Agrupar:
1. "A produção industrial está em alta este mês."
2. "Agricultores relatam uma colheita abundante nesta temporada."
3. "O crescimento econômico acelerou no último trimestre."

Agrupamento:
    Frases 1 e 3: Ambas tratam de crescimento econômico.
    Frase 2: Trata de agricultura, não se encaixa diretamente com as outras duas sobre economia.

Frases para Agrupar:
1. "Pesquisadores descobrem um novo tipo de antibiótico no solo da floresta tropical."
2. "A conferência de tecnologia revela inovações em inteligência artificial."
3. "Novo método de reciclagem aumenta a eficiência na conversão de resíduos plásticos."

Agrupamento:
    Frase 1: Relacionada a descobertas científicas.
    Frases 2 e 3: Relacionadas a inovações tecnológicas.


Q: 
Frases para Agrupar:

"As vendas de carros elétricos da Tesla continuam a subir apesar da pandemia"
"Novo estudo sugere que o consumo de café pode diminuir o risco de doenças cardíacas"
"O modelo mais recente do iPhone apresenta um

<hr>

################################## SUMARIZAÇÃO DE TEXTO ##################################


In [71]:
################# "World Knowledge" e Compreensão #################

context = """
Texto: "A água é essencial para a vida na Terra. Ela não apenas sustenta os organismos vivos, mas também participa de vários processos vitais, como o ciclo hidrológico, que afeta o clima global e as condições meteorológicas."
Pergunta: Resuma o texto.
Resposta: A água é crucial para a vida e processos globais como o clima.

Texto: "As florestas tropicais são ricas em biodiversidade. Elas não só abrigam uma grande variedade de espécies animais e vegetais, mas também ajudam a regular o clima terrestre e são vitais para a manutenção de muitos ecossistemas."
Pergunta: Resuma o texto.
Resposta: Florestas tropicais têm alta biodiversidade e são importantes para o clima e ecossistemas.
"""



question = """
Texto: "Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cada um de nós fez algo na juventude que nos envergonhará no futuro. Essas coisas podem ser como uma sombra que o acompanha pelo resto da vida, de modo que você só consegue olhar para ela. Mas o tempo não vai mudar, os seus melhores esforços para compensar isso, como não é uma espécie de auto-ajuda? A beleza e a amizade da infância, por causa de uma negligência covarde e arruinada, se você tiver outra chance, você está disposto a se esforçar para recuperar aquele antigo eu?
Mas as pessoas são assim, vivem sempre dentro de um determinado limite de tempo, onde o mundo pode estar alguns anos depois nem elas conseguem entender, mas é isso que não conseguimos superar. Para você, mil vezes, todo o corpo ainda estará justo, talvez isso seja a vida, a vida não é apenas fazer coisas que valem a pena!"

Pergunta: Resuma o texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Texto: "A água é essencial para a vida na Terra. Ela não apenas sustenta os organismos vivos, mas também participa de vários processos vitais, como o ciclo hidrológico, que afeta o clima global e as condições meteorológicas."
Pergunta: Resuma o texto.
Resposta: A água é crucial para a vida e processos globais como o clima.

Texto: "As florestas tropicais são ricas em biodiversidade. Elas não só abrigam uma grande variedade de espécies animais e vegetais, mas também ajudam a regular o clima terrestre e são vitais para a manutenção de muitos ecossistemas."
Pergunta: Resuma o texto.
Resposta: Florestas tropicais têm alta biodiversidade e são importantes para o clima e ecossistemas.


Q: 
Texto: "Sabemos onde realmente está a pipa em nosso coração? Não conseguiremos isso de novo se perdermos isso na vida, talvez nos arrependamos e nos redimamos, mas tudo isso parece ser tarde demais. Sempre que a pipa é lançada no céu, não deveríamos nos perguntar se realmente valorizamos o que temos?
Cad

In [72]:
################# Sumarização de Texto #################
################# "World Knowledge" e Compreensão #################

context = """
Texto: "A água é vital para todas as formas de vida conhecidas. A importância da água para os seres vivos é tão grande que ela é considerada uma substância essencial para a existência de vida."
Pergunta: Resuma o texto.
Resposta: A água é crucial para a vida de todos os seres vivos conhecidos.

Texto: "As florestas são essenciais para a saúde do nosso planeta. Elas fornecem oxigênio, abrigam biodiversidade, regulam o clima e oferecem recursos para a vida humana."
Pergunta: Resuma o texto.
Resposta: As florestas desempenham um papel vital no bem-estar do planeta e no suporte à vida.
"""

question = """
Texto:  "Escrever é esquecer. A literatura é a maneira mais agradável de ignorar a vida. 
A música embala, as artes visuais animam, as artes vivas (como a dança e a arte de representar) entretêm. 
A primeira, porém, afasta-se da vida por fazer dela um sono; as segundas, contudo, não se afastam da vida - umas porque usam de fórmulas visíveis e portanto vitais, outras porque vivem da mesma vida humana.
Não é o caso da literatura. Essa simula a vida. Um romance é uma história do que nunca foi e um drama é um romance dado sem narrativa. 
Um poema é a expressão de ideias ou de sentimentos em linguagem que ninguém emprega, pois que ninguém fala em verso."
Fernando Pessoa

Pergunta: Resuma o texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Texto: "A água é vital para todas as formas de vida conhecidas. A importância da água para os seres vivos é tão grande que ela é considerada uma substância essencial para a existência de vida."
Pergunta: Resuma o texto.
Resposta: A água é crucial para a vida de todos os seres vivos conhecidos.

Texto: "As florestas são essenciais para a saúde do nosso planeta. Elas fornecem oxigênio, abrigam biodiversidade, regulam o clima e oferecem recursos para a vida humana."
Pergunta: Resuma o texto.
Resposta: As florestas desempenham um papel vital no bem-estar do planeta e no suporte à vida.


Q: 
Texto:  "Escrever é esquecer. A literatura é a maneira mais agradável de ignorar a vida. 
A música embala, as artes visuais animam, as artes vivas (como a dança e a arte de representar) entretêm. 
A primeira, porém, afasta-se da vida por fazer dela um sono; as segundas, contudo, não se afastam da vida - umas porque usam de fórmulas visíveis e portanto vitais, outras porque vivem da mesma vida humana.


<hr>

########################################### TRANSFORMAÇÃO DE FORMATOS  ###########################################

In [73]:
################# Converter uma frase na terceira pessoa  para a primeira pessoa #################

context = """
Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa: Silva acredita em outra coisa. De acordo com sua pesquisa, as declarações anteriores sobre este assunto estão incorretas.
Resposta: Acredito em outra coisa, disse Silva. De acordo com a minha pesquisa, as declarações anteriores sobre este assunto estão incorretas.
"""

question = """
Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa:
Apesar dos desafios envolvidos, os pesquisadores continuam a defender as suas reivindicações.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=100,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa: Silva acredita em outra coisa. De acordo com sua pesquisa, as declarações anteriores sobre este assunto estão incorretas.
Resposta: Acredito em outra coisa, disse Silva. De acordo com a minha pesquisa, as declarações anteriores sobre este assunto estão incorretas.


Q: 
Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa:
Apesar dos desafios envolvidos, os pesquisadores continuam a defender as suas reivindicações.

A: 
Embora sejam envolvidos desafios, eu continuo a defender as minhas reivindicações.


Q: 
Transforme a frase, que está na terceira pessoa, em frase em primeira pessoa:
A ideia foi apresentada pela equipe de pesquisa.

A: 
A ideia foi apresentada pela equipe de pesquisa, mas


In [74]:
################# Machine Translation #################


context = """
Tales from Shakespeare is an English children's book written by Charles Lamb and his sister Mary Lamb in 1807. The book is designed to make the stories of Shakespeare's plays familiar to the young. However, as noted in the author's Preface, "[Shakespeare's] words are used whenever it seemed possible to bring them in; and in whatever has been added to give them the regular form of a connected story, diligent care has been taken to select such words as might least interrupt the effect of the beautiful English tongue in which he wrote: therefore, words introduced into our language since his time have been as far as possible avoided." 
"""

question = """
Traduza o parágrafo abaixo para o português brasileiro
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Tales from Shakespeare is an English children's book written by Charles Lamb and his sister Mary Lamb in 1807. The book is designed to make the stories of Shakespeare's plays familiar to the young. However, as noted in the author's Preface, "[Shakespeare's] words are used whenever it seemed possible to bring them in; and in whatever has been added to give them the regular form of a connected story, diligent care has been taken to select such words as might least interrupt the effect of the beautiful English tongue in which he wrote: therefore, words introduced into our language since his time have been as far as possible avoided." 


Q: 
Traduza o parágrafo abaixo para o português brasileiro

A: O parágrafo abaixo pode ser traduzido para o português brasileiro da seguinte maneira: "O livro Tales from Shakespeare, escrito por Charles Lamb e sua irmã Mary Lamb em 1807, tem como objetivo tornar as histórias das peças de Shakespeare conhecidas para as crianças. No entanto, como notado no 

In [75]:
################# TRANSORMAÇÃO DE FORMATOS  #################


context = """
As Seller, an encyclopedia salesman, approached the grounds on which the house of Hermit, the hermit, was situated, he saw a sign that said, "No salesmen. Trespassers will be prosecuted. Proceed at your own risk."
Although Seller had not been invited to enter, he ignored the sign and drove up the driveway toward the house. As he rounded a curve, a powerful explosive charge buried in the driveway exploded, and Seller was injured. 

"""

question = """
Traduza o parágrafo para o português.

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


As Seller, an encyclopedia salesman, approached the grounds on which the house of Hermit, the hermit, was situated, he saw a sign that said, "No salesmen. Trespassers will be prosecuted. Proceed at your own risk."
Although Seller had not been invited to enter, he ignored the sign and drove up the driveway toward the house. As he rounded a curve, a powerful explosive charge buried in the driveway exploded, and Seller was injured. 



Q: 
Traduza o parágrafo para o português.


A: 
Enquanto o Vendedor, um vendedor de enciclopédias, se aproximava dos terrenos em que se situava a casa do Ermitão, o hermita, viu uma sina que dizia: "Não se admitem vendedores. Intrusos serão processados. Procedam a seu risco."

Apesar de não ter sido convidado a entrar, ele ignorou a sina e seguiu pela estrada até à casa. Enquanto redonda uma curva, uma poderosa carga explosiva enterrada na estrada explodiu, e o Vendedor foi ferido.


In [76]:
################# TRANSORMAÇÃO DE FORMATOS E ICL com self-reflection  #################

context = """
"Um pedestre atravessou a rua fora da faixa de pedestres e foi atingido por um carro. O motorista estava acima do limite de velocidade."
Pergunta: O motorista é responsável pelos ferimentos do pedestre?
(A) Sim, porque ele estava acima do limite de velocidade.
(B) Não, porque o pedestre não estava na faixa.
Resposta: (A) Sim, porque ele estava acima do limite de velocidade.
Justificativa: Apesar de o pedestre não estar na faixa, o motorista tem a responsabilidade final de controlar a velocidade do veículo para evitar acidentes, o que é uma expectativa legal padrão.

"Um cliente escorrega e cai em uma loja onde um sinal de 'Piso Molhado' estava claramente visível."
Pergunta: A loja é responsável pelos ferimentos do cliente?
(A) Sim, porque é responsabilidade da loja manter um ambiente seguro.
(B) Não, porque o sinal de aviso estava visível.
Resposta: (B) Não, porque o sinal de aviso estava visível.
Justificativa: A presença de um aviso claro reduz a responsabilidade da loja, pois isso significa que o cliente foi devidamente informado sobre o perigo potencial.
"""

question = """
Can Seller recover damages from Hermit for his injuries?
(A) Yes, unless Hermit, when he planted the charge, intended only to deter, not harm, intruders.
(B) Yes, if Hermit was responsible for the explosive charge under the driveway.
(C) No, because Seller ignored the sign, which warned him against proceeding further.
(D) No, if Hermit reasonably feared that intruders would come and harm him or his family.

Justifique a escolha da alternativa correta entre as quatro apresentadas (A, B, C ou D).
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


"Um pedestre atravessou a rua fora da faixa de pedestres e foi atingido por um carro. O motorista estava acima do limite de velocidade."
Pergunta: O motorista é responsável pelos ferimentos do pedestre?
(A) Sim, porque ele estava acima do limite de velocidade.
(B) Não, porque o pedestre não estava na faixa.
Resposta: (A) Sim, porque ele estava acima do limite de velocidade.
Justificativa: Apesar de o pedestre não estar na faixa, o motorista tem a responsabilidade final de controlar a velocidade do veículo para evitar acidentes, o que é uma expectativa legal padrão.

"Um cliente escorrega e cai em uma loja onde um sinal de 'Piso Molhado' estava claramente visível."
Pergunta: A loja é responsável pelos ferimentos do cliente?
(A) Sim, porque é responsabilidade da loja manter um ambiente seguro.
(B) Não, porque o sinal de aviso estava visível.
Resposta: (B) Não, porque o sinal de aviso estava visível.
Justificativa: A presença de um aviso claro reduz a responsabilidade da loja, pois isso 

In [77]:
################# Retirar Referências do Texto #################

context = """

Esta é uma tarefa de retirada de referências bibliográficas do texto. As referências estão escritas no formato "[número]". Exemplo: [10].

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.
"""



question = """
Contexto: José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33] D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35] Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36] Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37] Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38] D. Pedro II não era um gênio,[39] mas inteligente[40] e com grande capacidade para acumular conhecimento facilmente.[41]

Remova do texeto as referências no formato "[número]" e escreva a Resposta:

"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)



Esta é uma tarefa de retirada de referências bibliográficas do texto. As referências estão escritas no formato "[número]". Exemplo: [10].

Exemplos:
Contexto: Joao da silva era carpinteiro [1][24]. Estudou na escola santa amelia [17] até os 18 anos de idade.
Resposta: Joao da silva era carpinteiro. Estudou na escola santa amelia até os 18 anos de idade.
Contexto: Ontem foi dia de ramos [32].
Resposta: Ontem foi dia de ramos.


Q: 
Contexto: José Bonifácio foi destituído de sua posição em dezembro de 1833 e substituído por outro tutor.[31][32][33] D. Pedro II passava os dias estudando,[34] com apenas duas horas livres para recreação.[35] Acordava às 06h30 da manhã e começava seus estudos às sete, continuando até as dez da noite, quando ia para cama.[36] Tomou-se grande cuidado em sua educação para formar valores e personalidade diferente da impulsividade e irresponsabilidade demonstradas por seu pai.[31][37] Sua paixão pela leitura lhe permitiu assimilar qualquer informação.[38] D. Pe

<hr>

############################################# EXTRAÇÃO DE CONTEÚDO  #############################################

In [78]:
################# Extração de Conteúdo #################

context = """
A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem \
por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. \
O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Pergunta: Qual é o maior sistema de recifes de corais do mundo?
Resposta: A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo.

Pergunta: Quantas espécies de peixes podem ser encontradas na Grande Barreira de Corais?
Resposta: Mais de 1.500 espécies de peixes podem ser encontradas na Grande Barreira de Corais.
"""

question = """
Dada a descrição do contexto acima:
O que é a Grande Barreira de Corais e onde ela está localizada?
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Geração da resposta
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=160,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, composto por mais de 2.900 recifes individuais e 900 ilhas que se estendem por mais de 2.300 quilômetros. Está localizado no Mar de Coral, na costa da Austrália. O recife abriga uma grande variedade de vida marinha, incluindo mais de 1.500 espécies de peixes, 600 tipos de corais e inúmeras outras espécies.

Pergunta: Qual é o maior sistema de recifes de corais do mundo?
Resposta: A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo.

Pergunta: Quantas espécies de peixes podem ser encontradas na Grande Barreira de Corais?
Resposta: Mais de 1.500 espécies de peixes podem ser encontradas na Grande Barreira de Corais.


Q: 
Dada a descrição do contexto acima:
O que é a Grande Barreira de Corais e onde ela está localizada?

A: A Grande Barreira de Corais é o maior sistema de recifes de corais do mundo, localizado no Mar de Coral, na costa da Austrália. Ele se estende por mais de 2.300 quilôm

In [79]:
################# Reconhecimento e Extração de Informações Específicas #################

context = """
Texto de Referência:
O Brasil é o quinto maior país do mundo em território, com uma área total de 8.515.767 quilômetros quadrados. É também o quinto país mais populoso do mundo, \
com uma população estimada em cerca de 211 milhões de pessoas em 2021. \
O país é conhecido por sua diversidade cultural e natural, incluindo a floresta Amazônica, que é a maior floresta tropical do mundo.
Pergunta: Qual a dimensão do Brasil em área?
Resposta: 8.515.767 quilômetros quadrados.

"""
question = """
Texto de Referência:
O Brasil é o quinto maior país do mundo em território, com uma área total de 8.515.767 quilômetros quadrados. É também o quinto país mais populoso do mundo, com uma população estimada em cerca de 211 milhões de pessoas em 2021. O país é conhecido por sua diversidade cultural e natural, incluindo a floresta Amazônica, que é a maior floresta tropical do mundo.
Pergunta: Qual é a posição do Brasil em termos de tamanho de território e população, e qual é a principal característica natural mencionada no texto?
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)



Texto de Referência:
O Brasil é o quinto maior país do mundo em território, com uma área total de 8.515.767 quilômetros quadrados. É também o quinto país mais populoso do mundo, com uma população estimada em cerca de 211 milhões de pessoas em 2021. O país é conhecido por sua diversidade cultural e natural, incluindo a floresta Amazônica, que é a maior floresta tropical do mundo.
Pergunta: Qual a dimensão do Brasil em área?
Resposta: 8.515.767 quilômetros quadrados.



Q: 
Texto de Referência:
O Brasil é o quinto maior país do mundo em território, com uma área total de 8.515.767 quilômetros quadrados. É também o quinto país mais populoso do mundo, com uma população estimada em cerca de 211 milhões de pessoas em 2021. O país é conhecido por sua diversidade cultural e natural, incluindo a floresta Amazônica, que é a maior floresta tropical do mundo.
Pergunta: Qual é a posição do Brasil em termos de tamanho de território e população, e qual é a principal característica natural mencionada 

In [80]:
################# Classificação de Intenções #################

context = """
Texto:
"Estou tendo problemas para fazer login na minha conta, e já tentei redefinir minha senha várias vezes sem sucesso. O que mais eu posso fazer para resolver isso?"
Intenção: Solicitar assistência para reaver acesso à conta.

Texto:
"Acabei de receber o produto que comprei de vocês, e estou muito satisfeito com a qualidade. Quero agradecer e dizer que continuarei sendo um cliente fiel."
Intenção: Expressar satisfação e gratidão.

"""

question = """
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João 

Classifique a intenção presente no texto.
"""


# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)



Texto:
"Estou tendo problemas para fazer login na minha conta, e já tentei redefinir minha senha várias vezes sem sucesso. O que mais eu posso fazer para resolver isso?"
Intenção: Solicitar assistência para reaver acesso à conta.

Texto:
"Acabei de receber o produto que comprei de vocês, e estou muito satisfeito com a qualidade. Quero agradecer e dizer que continuarei sendo um cliente fiel."
Intenção: Expressar satisfação e gratidão.



Q: 
Olá
Passei algum tempo analisando sua documentação, mas não consegui descobrir como adicionar um novo cartão de crédito.
É um problema porque meu cartão atual irá expirar em breve e temo que isso cause uma interrupção no serviço.
Como posso atualizar meu cartão de crédito?
Desde já, obrigado,
João 

Classifique a intenção presente no texto.

A: A intenção presente no texto é solicitar assistência para atualizar o cartão de crédito.


In [81]:
################# Reconhecimento e Extração de Entidades #################


context = """
Texto:
"A Microsoft, fundada por Bill Gates e Paul Allen, está sediada em Redmond, Washington."

Análise:
{
  "pessoa": ["Bill Gates", "Paul Allen"],
  "lugar": ["Redmond", "Washington"],
  "organização": ["Microsoft"]
}

"""

question = """
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.

Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Texto:
"A Microsoft, fundada por Bill Gates e Paul Allen, está sediada em Redmond, Washington."

Análise:
{
  "pessoa": ["Bill Gates", "Paul Allen"],
  "lugar": ["Redmond", "Washington"],
  "organização": ["Microsoft"]
}



Q: 
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em várias cores, apresenta um processador mais rápido e maior duração da bateria.

Analise o parágrafo, identifique em formato JSON Somente as entidades ("entities") dos tipos "pessoa", "lugar" e "organização" mencionadas no texto.

A:
{
  "pessoa": ["Tim Cook"],
  "lugar": ["Cupertino", "Califórnia"],
  "organização": ["Apple"]
}



Q:
O evento da Apple aconteceu no Steve Jobs Theatre em Cupertino, Califórnia. 
Tim Cook, CEO da Apple, apresentou o novo iPhone 15, que estará disponível para pré-encomenda a partir da próxima semana. 
O telefone vem em

In [82]:
################# Extração de Relações #################


context = """
Texto:
"Mark Zuckerberg criou o Facebook enquanto estudava em Harvard."
Análise de Relacionamento:
{
  "entidade1": "Mark Zuckerberg",
  "entidade2": "Facebook",
  "relacionamento": "criou"
}

Texto:
"Jeff Bezos fundou a Amazon em 1994 e transformou-a numa das maiores empresas de comércio eletrônico do mundo."
Análise de Relacionamento:
{
  "entidade1": "Jeff Bezos",
  "entidade2": "Amazon",
  "relacionamento": "fundou"
}

"""

question = """
Elon Musk fundou a SpaceX em 2002 com o objetivo de reduzir os custos de transporte espacial e possibilitar a colonização de Marte.

Analise a frase e extraia o relacionamento entre duas entidades e, em seguida, produza os resultados indicando: entities e relatiohships entre estas no formato JSON
"""

# Combinação do contexto e da questão para formar o prompt
prompt = context + "\n\nQ: " + question + "\nA:"

# Tokenização do prompt
tokens = tokenizer(prompt, return_tensors="pt")

# Extrair os tensores de input_ids e da attention_mask
input_ids = tokens["input_ids"]
input_ids = input_ids.to('cuda')
attention_mask = tokens["attention_mask"]

# Outputs - Passar ambos para o modelo e gerar a resposta (generate_ids = outputs)
with torch.no_grad():
    generate_ids = peft_model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=256,
        #do_sample=True,
        #top_k=40,
    )

# Decodificação e impressão da resposta (answer = last_hidden_states)
last_hidden_states = tokenizer.decode(generate_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
answer = last_hidden_states
print(answer)


Texto:
"Mark Zuckerberg criou o Facebook enquanto estudava em Harvard."
Análise de Relacionamento:
{
  "entidade1": "Mark Zuckerberg",
  "entidade2": "Facebook",
  "relacionamento": "criou"
}

Texto:
"Jeff Bezos fundou a Amazon em 1994 e transformou-a numa das maiores empresas de comércio eletrônico do mundo."
Análise de Relacionamento:
{
  "entidade1": "Jeff Bezos",
  "entidade2": "Amazon",
  "relacionamento": "fundou"
}



Q: 
Elon Musk fundou a SpaceX em 2002 com o objetivo de reduzir os custos de transporte espacial e possibilitar a colonização de Marte.

Analise a frase e extraia o relacionamento entre duas entidades e, em seguida, produza os resultados indicando: entities e relatiohships entre estas no formato JSON

A:
{
  "entidade1": "Elon Musk",
  "entidade2": "SpaceX",
  "relacionamento": "fundou",
  "objetivo": "reduzir os custos de transporte espacial e possibilitar a colonização de Marte"
}
